In this module we want to dive into process monitoring. While it is possible to use AI to do this, we want to use a more simple but yet effective approach. In order to get some hands on experience, we will use data from the IEEE PHM 2023 Prognostic challenge. The goal is to detect bearing degradation (so this is an anomaly detection problem). We have vibration signals (horizontal and vertical) for all bearings. We have a training set with data from 'normal' bearings, as well as a test set (with 'normal' and degraded bearings). We want to detect the faulty bearings in the test set.
The sensor sampling rate is 25.6 kHz. We will use a confidence level of 0.95 for the anomaly detection.

In [26]:
import numpy as np
import pickle
from matplotlib import pyplot as plt
from scipy.signal import hilbert, butter, filtfilt
import scipy.stats as st
import math



sampling_rate = 25600
SENSOR = 'horizontal'   # horizontal / vertical
CONFIDENCE_LVL = 0.95

First, we will load the training and test data.

In [ ]:
# Load training and test data
with open('train_data_pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('test_data_pkl', 'rb') as f:
    test_data = pickle.load(f)

for idx, data in enumerate(train_data):
    if SENSOR == 'horizontal':
        train_data[idx] = data[:,4]
    else:
        train_data[idx] = data[:,5]
for idx, data in enumerate(test_data):
    if SENSOR == 'horizontal':
        test_data[idx] = data[:,4]
    else:
        test_data[idx] = data[:,5]

print(f'Size training set: {len(train_data)}')
print(f'Size training set: {len(test_data)}')

Let us have a look at the training data. It should only contain data from good bearings that haven't degraded yet. Let us first look at data from a single bearing.

In [ ]:
# visualization of single bearing from training dataset
plt.plot(train_data[1], c='grey', alpha=0.5)
plt.show()

Now we will visualize the whole training dataset (500 bearings).

In [ ]:
# visualization of training data
for data_bad_part in train_data:
    plt.plot(data_bad_part, c='grey', alpha=0.5)
plt.show()

Looks good! The vibration sensor is not exactly the same for every bearing, but that is completely normal. Now let's look at the test data. Besides data from good bearings, the dataset should also contain data from degraded bearings.

In [ ]:
# visualization of test data
for data_bad_part in test_data:
    plt.plot(data_bad_part, c='grey', alpha=0.5)
plt.show()

The test set is significantly smaller, so we are seeing less extreme peaks. We will try to set up a system that is able to detect degraded bearings automatically. We will create envelopes using the training data. This allows us to detect anomalies (degraded bearings) by comparing test data to the envelopes. First, we will create some methods in order to create an envelope from the training data.
We are applying a bandpass filter for preprocessing the time series data. After that, we will use the hilbert transformation for creating an envelope. We are creating as many envelopes as we have bearings in the training dataset. In the last step, we combine all envelopes using the mean for each time step.

In [32]:
# bandpass filtering
def bandpass_filter(data, lowcut, highcut, order=4):
    b, a = butter(order, [lowcut, highcut], btype='band', fs=sampling_rate)
    return filtfilt(b, a, data)

def preprocessing(input_data):
    input_data_filtered = bandpass_filter(input_data, lowcut=1000, highcut=10000)
    return np.absolute(input_data_filtered)

# hilbert transformation
def create_single_envelope(input_signal):
    transformed_signal = hilbert(input_signal)
    envelope = np.abs(transformed_signal)
    return envelope

# train envelope from multiple curves
def train_envelope(n_parts):
    # check how many files in directory
    n_files = len(train_data)
    if n_parts > n_files:
        n_parts = n_files
        print(f'Only {n_files} files in directory!')
    # create single envelopes for every signal
    i = 0
    single_envelopes = []
    while i<n_parts:
        data = train_data[i]
        data_signal = preprocessing(data)
        env = create_single_envelope(data_signal)
        single_envelopes.append(env)
        i += 1
    # combine envelopes
    combined_envelope = np.mean(single_envelopes, axis=0)   # combine by mean value
    return combined_envelope

Now we need a method to check, if a bearing is good or not using the envelope we just created. We are calculate mean and standard deviation of the envelope and create a threshold value for good bearings. We can now compare the mean of a test bearing against the threshold value.

In [33]:
# check, if parts are good
def check_parts(data_test_parts, envelope):
    # calculate mean and standard deviation of envelope as reference
    mean_envelope = np.mean(envelope)
    std_envelope = np.std(envelope)
    # threshold value for good parts
    z = st.norm.ppf((1+CONFIDENCE_LVL)/2)
    threshold = mean_envelope + z*std_envelope
    
    data_bad_parts = []
    indexes_bad_parts = []
    # check test parts
    for idx, test_part in enumerate(data_test_parts):
        mean_test_part = np.mean(test_part)
        if mean_test_part > threshold:
            data_bad_parts.append(test_part)
            indexes_bad_parts.append(idx)
    return data_bad_parts, indexes_bad_parts

Now we can test our system using the training and test data.

In [ ]:
# create envelope
envelope = train_envelope(400)
# read test data
data_test_parts = []
for data in test_data:
    data = preprocessing(data)
    data_test_parts.append(data)

# check test parts
data_bad_parts, indexes_bad_parts = check_parts(data_test_parts, envelope)
print(f'{len(data_bad_parts)} bad parts were identified.')
print(f'Indexes of bad parts: {indexes_bad_parts}')

# visualization
for data_bad_part in data_bad_parts:
    plt.plot(data_bad_part, c='grey', alpha=0.5)
plt.plot(envelope, label='envelope', linestyle='--', c='red')
plt.legend()
plt.show()